# Different Structure Hybrid Recommender

In [ ]:
def init_kaggle(run_on_kaggle, secret_name):
    if run_on_kaggle:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        personal_token = user_secrets.get_secret(secret_name)
        
        import subprocess
        subprocess.run(["git", "clone", "https://" + personal_token + "@github.com/alecontuIT/rec_sys_challenge2022.git"])
        
        import os
        os.chdir("./rec_sys_challenge2022")
        
        subprocess.run(["pip", "install", "-r", "requirements.txt"])
        return True
    
    else:
        return False

In [ ]:
dataset_version = "interactions-summed-transformed"
kaggle = init_kaggle(False, "recsys_git_token")

In [ ]:
import utils
from recmodels import *
from Recommenders.Hybrids.DiffStructHybridRecommender import DiffStructHybridRecommender
from HyperparameterTuning.IterativeHybridOptimizer import DiffStructHybridOptimizer
from scipy.stats import loguniform
from Evaluation.Evaluator import EvaluatorHoldout
from skopt.space import Real, Integer, Categorical
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

In [ ]:
recommender_class = DiffStructHybridRecommender

In [ ]:
URM_all, URM_train, URM_val, ICM = utils.get_data_global_sample(dataset_version=dataset_version, 
                                                                train_percentage=0.7,
                                                                setSeed=True,
                                                                k=1,
                                                                transformation="tanh")

In [ ]:
#utils.generate_best_models_after_commit(URM_all, URM_train, URM_val, RP3BetaRec, dataset_version)

## Optimizer

In [ ]:
optimizer = DiffStructHybridOptimizer(URM_all,
                                      URM_train, 
                                      URM_val,  
                                      dataset_version, 
                                      not_trained_recs_classes=
                                      [#TopPopRec, 
                                       #ItemKNNCFRec, 
                                       #UserKNNCFRec, 
                                       #IALSRec,
                                       #SLIM_BPRRec, 
                                       P3AlphaRec, 
                                       RP3BetaRec, 
                                       EASE_R_Rec, 
                                       #MatrixFactorizationBPRRec, 
                                       #FunkSVDRec, 
                                       #AsySVDRec, 
                                       #PureSVDRec, 
                                       #PureSVDItemRec, 
                                       #ScaledPureSVDRec, 
                                       #SVDFeatureRec
                                       ], 
                                      trained_recs=[hybrid])


In [ ]:
optimizer.inverse_incremental_bayesian_search(4, 0.5)

In [ ]:
hybrid = DiffStructHybridRecommender(URM_val, 
                            recs_on_urm_splitted=True, 
                            dataset_version=dataset_version)

hybrid.load_model_by_version("UserKNNCFP3alpha")
#from Evaluation.Evaluator import EvaluatorHoldout
#evaluator_validation = EvaluatorHoldout(URM_val, cutoff_list=[10])
#evaluator_validation.evaluateRecommender(hybrid)
#utils.submission(hybrid, dataset_version)